In [1]:
! git init

Reinitialized existing Git repository in /Users/gaylejuntilla/MA3831_A3/.git/


In [13]:
!git add abc_news.ipynbb
!git commit -m "add final comments"

[main 7717188] Initialise abc_news.ipynb


In [14]:
!git push -u origin main

Enumerating objects: 12, done.
Counting objects: 100% (12/12), done.
Delta compression using up to 8 threads
Compressing objects: 100% (11/11), done.
Writing objects: 100% (11/11), 3.21 KiB | 3.21 MiB/s, done.
Total 11 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), done.
To https://github.com/gaylejuntilla/MA3831-A3.git
   cd64767..7717188  main -> main
branch 'main' set up to track 'origin/main'.


In [24]:
#pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [28]:
# pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 3.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
Note: you may need to restart the kernel to use updated packages.


In [58]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime, timezone, timedelta
from selenium.webdriver.chrome.service import Service

class ABCNewsCrawler:
    def __init__(self, start_url, cutoff_date="1 January 2025"):
        """Initialise the instance"""
        self.start_url = start_url
        self.cutoff_date = datetime.strptime(cutoff_date, "%d %B %Y")  # Convert string to datetime
        self.articles = pd.DataFrame(columns=['headline', 'title', 'date', 'content', 'url'])  # Store data
        
        # Initialize the service object for ChromeDriverManager
        service = Service(ChromeDriverManager().install())
        
        # Initialize WebDriver with service and options
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Run headless (no UI)
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        self.driver = webdriver.Chrome(service=service, options=options) 
        
    def run(self):
        """Runs the scraper: loads articles until cutoff date, extracts articles."""
        self.driver.get(self.start_url)
        time.sleep(3)  # Let page load
        
        # Step 1: Load articles until the cutoff date
        self.load_articles_until_cutoff()
        
        # Step 2: Extract articles after loading all
        self.extract_articles()
        
        self.driver.quit()  # Close browser
        #print(self.articles)

    def load_articles_until_cutoff(self):
        """Keep loading more articles until the last article is older than the cutoff date."""
    
        while True:
            # Get all currently loaded articles
            articles = self.driver.find_elements(By.CSS_SELECTOR, 'ul[aria-labelledby="Latest election articles"] li')

            if not articles:
                print("No articles found on page. Stopping.")
                break

            # Find the date of the last article
            try:
                # last article in list
                last_article = articles[-1] 
                # find date attribute
                last_date_str = last_article.find_element(By.XPATH, ".//time[1]").get_attribute("datetime")
                # convert to a datetime object
                last_article_date_utc = datetime.strptime(last_date_str, "%Y-%m-%dT%H:%M:%S.000Z")
                # convert to brisbane time
                last_article_date = self.convert_to_brisbane_time(last_article_date_utc)

                # Stop loading if the last article's date is older than the cutoff
                if last_article_date < self.cutoff_date:
                    print("Reached cutoff date. Stopping 'Load More' clicks.")
                    break

            except Exception as e:
                print(f"Error extracting date from last article: {e}")
                break  # Stop in case of unexpected errors

            # Try clicking "Load More" to get more articles
            try:
                load_more_button = WebDriverWait(self.driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-component="PaginationLoadMoreButton"]')))
                load_more_button.click()
                time.sleep(2)  # Wait for more articles to load

            except:
                print("No more 'Load More' button. Ending loading.")
                break
    
    def convert_to_brisbane_time(self, utc_datetime):
        """Converts a UTC datetime string to Brisbane time and formats it as 'DD Month YYYY'."""
    
        # Define UTC timezone
        utc_zone = timezone.utc

        # Define Brisbane timezone (UTC+10, no daylight savings)
        brisbane_zone = timezone(timedelta(hours=10))

        # Convert to Brisbane time
        brisbane_datetime = utc_datetime.replace(tzinfo=timezone.utc).astimezone(brisbane_zone)
        
        # Makes the date timezone independent, consistent with the other domains for processing
        brisbane_datetime_naive = brisbane_datetime.replace(tzinfo=None)
        
        # Format as 'DD Month YYYY'
        return brisbane_datetime_naive
    
    def extract_articles(self):
        """Extracts article details: headline, date, content, and URL."""
        
        # Extracts all article elements into a list
        articles = self.driver.find_elements(By.CSS_SELECTOR, 'ul[aria-labelledby="Latest election articles"] li')

        for article in articles:
            try:
                
                # Extract date
                article_date_str = article.find_element(By.XPATH, ".//time[1][@datetime]").get_attribute('datetime')

                # Convert the UTC datetime string to a datetime object
                article_date_utc = datetime.strptime(article_date_str, "%Y-%m-%dT%H:%M:%S.000Z")

                # Convert to UTC+10 and format it to just Day, Month, and Year
                article_date = self.convert_to_brisbane_time(article_date_utc)
                
                # Stop if we've reached the cutoff date
                if article_date < self.cutoff_date:
                    break
                    
                # Extract the article URL
                link_element = article.find_element(By.TAG_NAME, 'a')
                article_url = link_element.get_attribute("href")

                # Extract headline
                headline = article.find_element(By.CSS_SELECTOR, "a[data-component='Link']").text 
                
                # Extract content
                self.driver.execute_script("window.open('');")  # Open new tab
                self.driver.switch_to.window(self.driver.window_handles[1])
                self.driver.get(article_url)
                time.sleep(3) # time to load article
                
                # title is different from the headline so extract title as well
                title = self.driver.find_element(By.CSS_SELECTOR, "h1[data-component='Typography']").text

                content_elements = self.driver.find_elements(By.CSS_SELECTOR, 'div[class*="ArticleRender_article"] p')
                content = " ".join([p.text for p in content_elements])

                # Close tab and switch back
                self.driver.close()
                self.driver.switch_to.window(self.driver.window_handles[0])

                # Add data to DataFrame
                self.articles.loc[len(self.articles)] = [headline, title, article_date, content, article_url]
                print(f"Collected: {headline}")
                
                # wait 10 seconds between each article
                time.sleep(10)

            except Exception as e:
                print(f"Error processing article: {e}")

    
    def save_data_to_file(self):
        """Saves the scraped data to a CSV file."""
        self.articles.to_csv("abc_news_articles.csv", index=False)
        print("Data saved to abc_news_articles.csv.")


In [59]:
mycrawler = ABCNewsCrawler("https://www.abc.net.au/news/elections/federal-election-2025")

In [60]:
mycrawler.run()

Reached cutoff date. Stopping 'Load More' clicks.
Collected: How a 'chopper on a stick' entered the federal election campaign
Collected: Teal candidate banned from hair salon over 'poor attempt at humour'
Collected: Posters of politicians and billionaires as Nazis slammed as 'vile'
Collected: Albanese won't budge on key US grievances, as Trump's 'liberation day' looms
Collected: Coalition promises to relax home lending rules, against regulator's urging
Collected: Bipartisan support to invest $150m for health training hub in SA
Collected: 'How can I survive?' Politicians confronted by voters doing it tough on Q+A
Collected: Bob Brown Foundation launches legal challenge against PM's 'rushed' salmon bill
Collected: Albanese says PM should live in Canberra as Dutton opts for Kirribilli — as it happened
Collected: Young Australians could contribute to a 'Trump bump' this election
Collected: Albanese announces joint $355m upgrade to Perth hospital
Collected: Examining Labor's promise to ban 

Collected: Cost of living and housing: Key voting issues for NSW in upcoming election
Collected: Voters were convinced by our AI robocall about the election. It cost $100
Collected: Crossbenchers say hung parliament would have to negotiate bill by bill
Collected: Strong Labor city faces independent challenge in federal election
Collected: Cost of living, health, energy, among key voter concerns this federal election
Collected: Lawyer allegedly assaulted amid battle for Nationals stronghold
Collected: 'If you think it's biased, it is biased': Albanese and Dutton turn to podcasts ahead of election
Collected: Federal parties vie over ocean protection ahead of election
Collected: Legendary ABC chief election analyst Antony Green to step down
Collected: New bulk-billing funding won't sway this GP clinic
Collected: Labor pledges to expand voice and SMS coverage across Australia
Collected: Senator threatens climate body after nuclear 'carbon bomb' report
Collected: Dutton paints Labor as big 

In [61]:
# make sure to save
mycrawler.save_data_to_file()

Data saved to abc_news_articles.csv.


In [66]:
display(mycrawler.articles)

,headline,title,date,content,url
0,How a 'chopper on a stick' entered the federal...,Deputy PM promises return of Nowra's chopper o...,2025-04-01 18:18:43,An Iroquois 894 helicopter that stood proudly ...,https://www.abc.net.au/news/2025-04-01/nowra-c...
1,Teal candidate banned from hair salon over 'po...,Teal candidate banned from Sydney salon after ...,2025-04-01 17:32:15,Teal independent candidate Nicolette Boele has...,https://www.abc.net.au/news/2025-04-01/teal-in...
2,Posters of politicians and billionaires as Naz...,"McCormack slams posters of himself, Dutton, Ri...",2025-04-01 16:33:58,Former Deputy Prime Minister Michael McCormack...,https://www.abc.net.au/news/2025-04-01/former-...
3,"Albanese won't budge on key US grievances, as ...","Albanese won't budge on key US grievances, as ...",2025-04-01 10:42:21,An updated list of grievances published by the...,https://www.abc.net.au/news/2025-04-01/electio...
4,Coalition promises to relax home lending rules...,Coalition promises to relax home lending rules...,2025-04-01 18:04:02,The Coalition is promising to relax rules arou...,https://www.abc.net.au/news/2025-04-01/electio...
...,...,...,...,...,...
159,Community desperate for investment welcomes $3...,'Forgotten' Wyndham welcomes federal pledge to...,2025-01-11 09:21:14,Community leaders in Western Australia's north...,https://www.abc.net.au/news/2025-01-11/albanes...
160,A week of signals for the wild way politics mi...,Albanese's Bruce Highway announcement got as a...,2025-01-11 04:00:00,If you were constructing your federal election...,https://www.abc.net.au/news/2025-01-11/albanes...
161,Albanese looks revived in the heat as he reset...,From 'no Australian left behind' to 'no Austra...,2025-01-10 11:11:43,It's north of 40 degrees Celsius. The humidity...,https://www.abc.net.au/news/2025-01-10/albanes...
162,Albanese enters this election 'kicking against...,Albanese is running at the election 'kicking a...,2025-01-10 04:00:00,Anthony Albanese could have spent Christmas at...,https://www.abc.net.au/news/2025-01-10/albanes...
